# Part 3 of the Investigation into "their kingdom" of 4Q246 II.2

<img src="images/their_kingdom.png"></img>

In [1]:
from datetime import datetime
print(f'Notebook last updated on {datetime.now().__str__()}')

Notebook last updated on 2017-05-20 22:18:30.863896


# Introduction

The two previous notebooks, `their_kingdom_1.ipynb` and `their_kingdom_2.ipynb` addressed...

In [2]:
# begin code

# import tools
import collections, csv

# import text fabric
from tf.fabric import Fabric

# instantiate TF processor
TF = Fabric(modules='hebrew/etcbc4c', silent=True)

# load TF features
api = TF.load('''
              book chapter verse
              lex prs prs_ps
              ''')

# prevent having to prefix all functions with .api
api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.01s B book                 from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.01s B chapter              from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.01s B verse                from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.14s B lex                  from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.11s B prs                  from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B prs_ps               from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.00s Feature overview: 103 for nodes; 5 for edges; 1 configs; 7 computed
  5.13s All features loaded/computed - for details use loadLog()


In [3]:
kingdom = tuple(wd for wd in F.otype.s('word')
                  if F.lex.v(wd) in {'MMLKH/','MLKWT/'}
                  and F.prs.v(wd) not in {'NA','absent'}
                  and F.prs_ps.v(wd) == 'p3'
               )

print(len(kingdom))

37
